In [93]:
from random import sample
from time import time
import pandas as pd
import pymongo
from sklearn import ensemble
import numpy as np
import os
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.metrics import mean_squared_error
from math import sqrt

import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

<h3><u>CONSTANTS AND HELPER FUNSTIONS</u></h3>

In [22]:
query_collection = "macav2"
mongo_url = "mongodb://lattice-100:27018/"
mongo_db_name = "sustaindb"
query_fild = "gis_join"
sample_percent = 0.1
train_test = 0.8
feature_importance_percentage = 98
exhaustive_sample_percent = 0.0001


training_labels = ["min_surface_downwelling_shortwave_flux_in_air", "max_surface_downwelling_shortwave_flux_in_air",
                   "max_specific_humidity", "min_max_air_temperature", "max_max_air_temperature"]
target_labels = ["max_min_air_temperature"]

# QUERY-RELATED
sustainclient = pymongo.MongoClient(mongo_url)
sustain_db = sustainclient[mongo_db_name]

# QUERY projection
client_projection = {}
for val in training_labels:
    client_projection[val] = 1
for val in target_labels:
    client_projection[val] = 1
    
    
def fancy_logging(msg, unique_id=""):
    print(unique_id, ":", "====================================")
    print(unique_id, ":", msg, ": TIME: ",time())


<h3><u>DATA FETCH</u></h3>

In [3]:
# ACTUAL QUERYING
def query_sustaindb(query_gisjoin):

    sustain_collection = sustain_db[query_collection]
    client_query = {query_fild: query_gisjoin}

    start_time = time()
    query_results = list(sustain_collection.find(client_query, client_projection))
    
    return list(query_results)

def queryall_sustaindb():

    sustain_collection = sustain_db[query_collection]
    client_query = {}

    start_time = time()
    query_results = list(sustain_collection.find(client_query, client_projection))
    
    return list(query_results)

In [4]:
#df = query_sustaindb('G3701310')
df = queryall_sustaindb()
print("1: ", len(df))

1:  11353524


<h3><u>DATA SAMPLING</u></h3>

In [5]:
def data_sampling(query_results, exhaustive, sample_percent=1):
    if exhaustive:
        all_data = query_results
    else:
        data_size = int(len(query_results) * sample_percent)
        all_data = sample(query_results, data_size)

    return pd.DataFrame(all_data)

In [6]:
# RIKI
sampled_df = data_sampling(df, False, exhaustive_sample_percent)

In [7]:
Y = sampled_df.loc[:,target_labels]
X = sampled_df.loc[:, training_labels]
print(X.shape, Y.shape)

(1135, 5) (1135, 1)


<h3><u>DATA SPLITTING INTO TRAING AND VALIDATION</u></h3>

In [8]:
'''def data_partitioning(query_results, exhaustive, sample_percent=1):
    if exhaustive:
        all_data = query_results
    else:
        data_size = int(len(query_results) * sample_percent)
        all_data = sample(query_results, data_size)

    msk = np.random.rand(len(all_data)) < train_test

    all_data = pd.DataFrame(all_data)
    training_data = all_data[msk]
    val_data = all_data[~msk]
    return (pd.DataFrame(training_data), pd.DataFrame(val_data))'''

'def data_partitioning(query_results, exhaustive, sample_percent=1):\n    if exhaustive:\n        all_data = query_results\n    else:\n        data_size = int(len(query_results) * sample_percent)\n        all_data = sample(query_results, data_size)\n\n    msk = np.random.rand(len(all_data)) < train_test_split\n\n    all_data = pd.DataFrame(all_data)\n    training_data = all_data[msk]\n    val_data = all_data[~msk]\n    return (pd.DataFrame(training_data), pd.DataFrame(val_data))'

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


(908, 5) (227, 5) (908, 1) (227, 1)


<h3><u>MODELING</u></h3>

In [ ]:
'''parameters = [ {'n_estimators': 500, 'max_depth': 2, 'min_samples_split': 20},
                {'n_estimators': 300, 'max_depth': 3, 'min_samples_split': 50},
              {'n_estimators': 500, 'max_depth': 3, 'min_samples_split': 20},
              {'n_estimators': 600, 'max_depth': 3, 'min_samples_split': 15}]
    
for params in parameters:
    print("PARAMETERS:",params)
    count = 0
    error = 0
    for i in range(0,5):
        print("ROUND:",i)
        clf = ensemble.RandomForestRegressor(**params)
        clf.fit(X_train, pd.Series.ravel(y_train))

        rmse = sqrt(mean_squared_error(pd.Series.ravel(y_test), clf.predict(X_test)))
        print(rmse)
        error = error + rmse

        feature_importance = clf.feature_importances_
        feature_importance = 100.0 * (feature_importance / feature_importance.sum())
        sorted_idx = np.argsort(feature_importance)
        count = count+1
        print(np.flip(sorted_idx), np.flip(feature_importance[sorted_idx]))
    
    print("===============================================================")

    print("AVG RMSE:",(error/count))'''

# BETTER ALTERNATIVE BELOW

In [13]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
param_grid = {'max_depth': [2, 3], 'min_samples_split': [15, 20, 50]}
base_est = ensemble.RandomForestRegressor(random_state=0)
sh = HalvingGridSearchCV(base_est, param_grid, cv=5, verbose=3, 
                         factor=2, resource='n_estimators', max_resources=600).fit(X, pd.Series.ravel(Y))

n_iterations: 3
n_required_iterations: 3
n_possible_iterations: 3
min_resources_: 150
max_resources_: 600
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 6
n_resources: 150
Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END max_depth=2, min_samples_split=15, n_estimators=150;, score=(train=0.880, test=0.845) total time=   0.4s
[CV 2/5] END max_depth=2, min_samples_split=15, n_estimators=150;, score=(train=0.884, test=0.874) total time=   0.3s
[CV 3/5] END max_depth=2, min_samples_split=15, n_estimators=150;, score=(train=0.880, test=0.876) total time=   0.6s
[CV 4/5] END max_depth=2, min_samples_split=15, n_estimators=150;, score=(train=0.881, test=0.901) total time=   0.2s
[CV 5/5] END max_depth=2, min_samples_split=15, n_estimators=150;, score=(train=0.869, test=0.866) total time=   0.2s
[CV 1/5] END max_depth=2, min_samples_split=20, n_estimators=150;, score=(train=0.880, test=0.845) total time=   0.2s
[CV 2/5] END max_depth=2, min_samp

In [14]:
# THE BEST MODEL
clf_best = sh.best_estimator_

In [15]:
rmse = sqrt(mean_squared_error(pd.Series.ravel(y_test), clf_best.predict(X_test)))
rmse

3.3194087807191863

<h3><u>EXTRACT TOP FEATURES</u></h3>

In [17]:
feature_importance = clf_best.feature_importances_
feature_importance = 100.0 * (feature_importance / feature_importance.sum())
sorted_idx = np.argsort(feature_importance)
print(np.flip(sorted_idx), np.flip(feature_importance[sorted_idx]))

feature_importance = np.flip(feature_importance[sorted_idx])
sorted_idx=np.flip(sorted_idx)

print(sorted_idx)
print(feature_importance)


[2 4 3 0 1] [9.54214074e+01 4.38867615e+00 1.88880199e-01 1.03620492e-03
 0.00000000e+00]
[2 4 3 0 1]
[9.54214074e+01 4.38867615e+00 1.88880199e-01 1.03620492e-03
 0.00000000e+00]


<h3><u>FIND N FOR WHICH IMPORTANCE % > feature-importance-percentage</u></h3>

In [18]:
def find_cumulative(lists, val_max):
    cu_list = []
    length = len(lists)
    cu_list = [sum(lists[0:x:1]) for x in range(1, length+1)]
    
    print(cu_list)
    res = next(x for x, val in enumerate(cu_list)
                                  if val > val_max)
    return res

In [31]:
cut_off_indx = find_cumulative(feature_importance, feature_importance_percentage)

print("LAST INDEX: ", cut_off_indx)

[95.4214074454134, 99.8100835960911, 99.9989637950784, 100.0, 100.0]
LAST INDEX:  1


In [33]:
chopped_indices = sorted_idx[0:cut_off_indx+1]

print(sorted_idx)
print(chopped_indices)

[2 4 3 0 1]
[2 4]


<h3><u>SELECTED TOP COLUMNS</u></h3>

In [41]:
candidate_x_columns = list(X.columns)
candidate_y_columns = list(Y.columns)

print(candidate_x_columns)
print(candidate_y_columns)

['min_surface_downwelling_shortwave_flux_in_air', 'max_surface_downwelling_shortwave_flux_in_air', 'max_specific_humidity', 'min_max_air_temperature', 'max_max_air_temperature']
['max_min_air_temperature']


In [43]:
selected_x_columns = [candidate_x_columns[i] for i in chopped_indices]
selected_x_columns

['max_specific_humidity', 'max_max_air_temperature']

<b><hr /></b>

<h1><u><b>TRAINING PHASE #2</b></u></h1>

<h3><u>AGGREGATE QUERY OVER THE CHOSEN COLUMNS PER GIS-JOIN</u></h3>

In [74]:
'''sustain_collection = sustain_db[query_collection]
pipeline=[
   { "$project": { 'gis_join': '$gis_join', 'max_specific_humidity': '$max_specific_humidity', 'max_max_air_temperature': '$max_max_air_temperature', 'max_min_air_temperature': '$max_min_air_temperature'}},
   { "$group": { '_id': "$gis_join", 
"avg_max_specific_humidity": { "$avg": "$max_specific_humidity" },
"avg_max_max_air_temperature": { "$avg": "$max_max_air_temperature" },
"avg_max_min_air_temperature": { "$avg": "$max_min_air_temperature" }
  } }
]
cur = sustain_collection.aggregate(pipeline)

results = list(cur)
len(results)
'''

In [78]:
chopped_projection = []
chopped_projection.extend(selected_x_columns)
chopped_projection.extend(candidate_y_columns)

print(chopped_projection)

['max_specific_humidity', 'max_max_air_temperature', 'max_min_air_temperature']


In [80]:
def construct_chopped_query(chopped_projection, gis_join):
    # PROJECTION
    proj_d = {}
    proj_dict = {'$project': proj_d}
    
    #GROUP + AGGREGATION
    group_d = {}
    group_dict = {'$group': group_d}
    
    full_query=[proj_dict, group_dict]
    
    # PROJECTION PART
    for cp in chopped_projection:
        proj_d[cp] = "$"+str(cp)
    proj_d[gis_join] = "$"+str(gis_join)
    
    # GROUP PART
    group_d['_id'] = "$"+str(gis_join)
    for cp in chopped_projection:
        inner_dict = {}
        inner_dict["$avg"] = "$"+str(cp)
        group_d[cp] = inner_dict
    
    return full_query
    
    

In [82]:
agg_pipeline = construct_chopped_query(chopped_projection, query_fild)

In [83]:
cur = sustain_collection.aggregate(agg_pipeline)
agg_results = list(cur)

print(len(agg_results))

3108


In [85]:
agg_results[0]

{'_id': 'G1303070',
 'max_specific_humidity': 0.010248015329865863,
 'max_max_air_temperature': 298.66463618943334,
 'max_min_air_temperature': 285.35960279222553}

<h3><u>DATA STAGING FOR PHASE 2</u></h3>

In [88]:
phase2_df = pd.DataFrame(agg_results)

In [95]:
chopped_projection

['max_specific_humidity', 'max_max_air_temperature', 'max_min_air_temperature']

In [96]:
df_importance = phase2_df.loc[:, chopped_projection]
df_importance

,max_specific_humidity,max_max_air_temperature,max_min_air_temperature
0,0.010248,298.664636,285.359603
1,0.009536,295.464063,282.961655
2,0.011345,301.200444,287.548820
3,0.005304,292.316465,276.215785
4,0.009168,297.007123,284.217233
...,...,...,...
3103,0.008729,295.586985,283.527293
3104,0.008122,295.232022,281.684612
3105,0.006237,287.522124,276.567473
3106,0.006625,290.181618,280.437066


<h3><u>K-MEANS CLUSTERING</u></h3>

In [94]:
num_clusters = int(sqrt(len(agg_results)))

num_clusters

55

In [101]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')


0.016077470572132494
306.7446375581714
295.760583082398


In [97]:
kmeans = KMeans(n_clusters=num_clusters).fit(df_importance)
centroids = kmeans.cluster_centers_
print(centroids)

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-3.40282347e+38 -3.40282347e+38 -3.40282347e+38]
 [-1.84467441e+20 -1.84467441e+20 -1.84467441e+20]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+

/s/parsons/b/others/sustain/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (55). Possibly due to duplicate points in X.
  """Entry point for launching an IPython kernel.


In [126]:
from sklearn.metrics import pairwise_distances_argmin_min

df_ultimate = pd.DataFrame(columns=["gis_join", "cluster_id", "distance"])



for index, row in phase2_df.iterrows():
    input_x = row[chopped_projection]
    gis_join = row['_id']
    #print(input_x, gis_join)
    closest, d = pairwise_distances_argmin_min([np.array(input_x)], centroids)
    df_ultimate.loc[index] = [gis_join, closest[0], d[0]]
    
print(df_ultimate)



      gis_join cluster_id    distance
0     G1303070          0  413.074652
1     G4701150          0  409.104278
2     G1201210          0  416.420498
3     G0800010          0  402.174186
4     G3701070          0  411.087176
...        ...        ...         ...
3103  G0500750          0  409.584414
3104  G0500490          0  408.054123
3105  G5500390          0  398.946787
3106  G5300570          0  403.547171
3107  G3901350          0  403.052772

[3108 rows x 3 columns]


In [125]:
df_ultimate.to_csv("/tmp/clusters.csv")